## O codigo em si.

In [ ]:
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


df = pd.read_json('/content/TelecomX_Data.json')

print(df.head())

In [ ]:
#Normalização dos dados em Json em etapas.

df_customer = pd.json_normalize(df['customer'])
df_phone = pd.json_normalize(df['phone'])
df_internet = pd.json_normalize(df['internet'])
df_account = pd.json_normalize(df['account'])

#Tratamento dos dados

df_final = pd.concat([df[['customerID', 'Churn']], df_customer, df_phone, df_internet, df_account], axis=1)
df_final['Contas_Diarias'] = (df_final['Charges.Monthly'] / 30).round(1)
df_final.dropna(subset=['Charges.Total'], inplace=True)

yes_no_cols = []
for col in df_final.columns:
    if df_final[col].dtype == 'object':
        unique_values = df_final[col].unique()
        if all(val in ['Yes', 'No', np.nan, 'No internet service', 0, 1] for val in unique_values):
            if col not in ['gender', 'InternetService', 'Contract', 'PaymentMethod', 'MultipleLines']:
                 yes_no_cols.append(col)
            elif col == 'MultipleLines' and all(val in ['Yes', 'No', 'No phone service', np.nan, 0, 1] for val in unique_values):
                 yes_no_cols.append(col)

for col in yes_no_cols:
    df_final[col] = df_final[col].replace({'Yes': 1, 'No': 0})

cols_to_replace_zero = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'InternetService', 'MultipleLines', 'PhoneService']
for col in cols_to_replace_zero:
     if col in df_final.columns:
         df_final[col] = df_final[col].replace({'No internet service': 0, 'No phone service': 0, 'Yes': 1, 'No': 0})

cols_to_int = ['SeniorCitizen', 'Partner', 'Dependents','PhoneService','PaperlessBilling', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'MultipleLines']
for col in cols_to_int:
    if col in df_final.columns:
         df_final[col] = pd.to_numeric(df_final[col], errors='coerce').astype('int64')

df_final['Charges.Total'] = pd.to_numeric(df_final['Charges.Total'], errors='coerce')

display(df_final.head())

In [ ]:
#Correção de linhas em branco

df_final = df_final.replace('', np.nan)
df_final.dropna(inplace=True)
display(df_final.describe())

In [ ]:
#Primeiro grafico, optei por chamar as bibliotecas logo no começo para deixar a visualização do código mais limpa.

plt.figure(figsize=(6, 4))
ax = sns.countplot(data=df_final, x='Churn')
plt.title('Distribuição de Clientes em Churn')
plt.xlabel('Churn')
plt.ylabel('Número de Clientes')

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 5), textcoords='offset points')

plt.xticks([0, 1], ['Não', 'Sim'])

plt.show()

In [ ]:
#Segunda sequencia de graficos, demandou mais tempo para ser feito mas consegui fazer aos poucos.

categorical_cols = ['gender', 'MultipleLines', 'InternetService', 'Contract', 'PaymentMethod']

title_map = {
    'gender': 'Clientes por Genero',
    'MultipleLines': 'Clientes por Linhas Telefonicas',
    'InternetService': 'Clientes por Serviço de Internet',
    'Contract': 'Clientes por Contrato',
    'PaymentMethod': 'Clientes por Metodo de Pagamento'
}

xlabel_map = {
    'gender': 'Genero',
    'MultipleLines': 'Linhas Telefonicas',
    'InternetService': 'Serviço de Internet',
    'Contract': 'Contrato',
    'PaymentMethod': 'Metodo de Pagamento'
}


for col in categorical_cols:
    plt.figure(figsize=(8, 5))
    ax = sns.countplot(data=df_final, x=col, hue='Churn')

    plt.title(title_map[col])
    plt.xlabel(xlabel_map[col])
    plt.ylabel('Número de Clientes')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    for p in ax.patches:
        if p.get_height() > 0:
            ax.annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()),
                        ha='center', va='center', xytext=(0, 5), textcoords='offset points')

    handles, labels = ax.get_legend_handles_labels()
    new_labels = ['Não', 'Sim']
    ax.legend(handles, new_labels, title='Churn', bbox_to_anchor=(1.05, 1), loc='upper left')

    current_labels = [item.get_text() for item in ax.get_xticklabels()]
    new_xticklabels = current_labels


    if col == 'PaymentMethod':
        new_xticklabels = [label.replace(' (automatic)', '') for label in new_xticklabels]
        new_xticklabels = [label.replace('Mailed check', 'Cheque pelos Correios') for label in new_xticklabels]
        new_xticklabels = [label.replace('Electronic check', 'Cheque eletrônico') for label in new_xticklabels]
        new_xticklabels = [label.replace('Credit card', 'Cartão de credito') for label in new_xticklabels]
        new_xticklabels = [label.replace('Bank transfer', 'Transferencia bancaria') for label in new_xticklabels]

    elif col == 'MultipleLines':
        new_xticklabels = [label.replace('No phone service', 'Sem serviço telefônico') for label in new_xticklabels]
        new_xticklabels = [label.replace('0', 'Não').replace('1', 'Sim') for label in new_xticklabels]

    elif col == 'InternetService':
         new_xticklabels = [label.replace('0', 'Sem serviço de internet') for label in new_xticklabels]
         new_xticklabels = [label.replace('Fiber optic', 'Fibra Otica') for label in new_xticklabels]

    elif col == 'Contract':
         new_xticklabels = [label.replace('One year', '1 Ano') for label in new_xticklabels]
         new_xticklabels = [label.replace('Month-to-month', 'Mensal') for label in new_xticklabels]
         new_xticklabels = [label.replace('Two year', '2 Anos') for label in new_xticklabels]


    ax.set_xticklabels(new_xticklabels)

    plt.show()

In [ ]:
#Grafico final de correlação.

plt.figure(figsize=(8, 5))
ax1 = sns.histplot(data=df_final, x='Charges.Total', hue='Churn', kde=True, multiple='stack', bins=10)
plt.title('Total pago by Churn')
plt.xlabel('Total Pago')
plt.ylabel('Número de Clientes')
plt.legend(title='Churn', labels=['Sim', 'Não'])
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)

for container in ax1.containers:
    ax1.bar_label(container, fmt='%d', label_type='center')

plt.show()

plt.figure(figsize=(8, 5))
ax2 = sns.histplot(data=df_final, x='tenure', hue='Churn', kde=True, multiple='stack', bins=10)
plt.title('Tempo de contrato por Churn')
plt.xlabel('Contrato (Meses)')
plt.ylabel('Número de Clientes')
plt.legend(title='Churn', labels=['Sim', 'Não'])
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)

for container in ax2.containers:
    ax2.bar_label(container, fmt='%d', label_type='center')

plt.show()

##1.	Introdução
O objetivo dos gráficos e analises que poderão ser vistos a seguir, servirão de base para a pergunta principal que esse relatório busca responder. Quais clientes estão deixando nossos serviços e o que eles consomem.


## 2. Limpeza e Tratamento de Dados

Inicialmente, os dados foram carregados a partir de um arquivo JSON. Como padrão de arquivos JSON os dados aninhados em formato de dicionário (customer, phone, internet, account). Para facilitar a análise, as colunas aninhadas foram normalizadas e mescladas ao DataFrame (df_final). As colunas originais aninhadas foram removidas.
Durante a inspeção inicial, os espaços vazios encontrados foram tratados como valores ausentes. E esses foram removidos do DataFrame para garantir a integridade da análise.
A partir dai passei para a  conversão de colunas categóricas binárias ('Yes'/'No') para representações numéricas (1 e 0) para torná-las adequadas para análises e modelagens futura. Algumas colunas numéricas foram convertidas para o tipo de dado apropriado (int64, float64).


## 3. Análise Exploratória

Aqui foi modelado os padrões que dos clientes que poderiam estar associados à evasão (Churn). Verificando a distribuição de variáveis de categoria e numéricas em relação ao status de evasão.

Distribuição de evasão:

Grafico inicial dos clientes que cancelaram e não cancelaram.

**Análise de Variáveis Categóricas:**

Graficos que relacionavam a evazão com as variaveis do DataFrame, como gênero, serviço de internet, tipo de contrato e método de pagamento.

**Análise de Variáveis Numéricas:**

Foi analisado a distribuição de variáveis numéricas contínuas, como o total gasto pelo cliente e o tempo de contrato (tenure), em relação ao Churn.


## 4. Conclusões e Insights

Conforme se vê nos dados e graficos, podemos retirar alguns insights que mostram a evasão de clientes:

*   **Contrato Mensal:** Clientes que fazem contratos mensais tem uma taxa de evasão muito superior em comparação com aqueles com contratos de um ou dois anos. Mostrando que a fidelização dos cliente aumenta conforme a duração do contrato.
*   **Serviço de Internet Fibra Óptica:** CClientes de fibra óptica também tem uma tendência maior a cancelar. Isso indica problemas de qualidade, preço ou suporte associados a este serviço.
*   **Método de Pagamento:** Aqueles que utilizam cheque eletrônico como método de pagamento tem uma taxa de evasão mais alta. Isso pode estar relacionado à conveniência ou a problemas com este método de pagamento.
*   **Tempo de Contrato (Tenure):** Quando falamos de contrato (tenure) podemos observar que aqueles com menor tempo de serviço são mais propensos a cancelar. Enquanto a evasão diminui conforme o tempo de contrato aumenta.
*   **Total Gasto (Charges.Total):** Quanto total gasto clientes com gastos totais mais baixos tem uma maior propensão à evasão, especialmente nos primeiros meses.
meses.

Conforme pode ser visto a evasão não está ligada a um único fator, mas sim a uma combinação como: serviços utilizados, tipo de contrato e histórico de relacionamento com a empresa.

## 5. Recomendações

Com base no que foi visto acima, seguem as seguintes recomendações para reduzir o churn:

*   **Contratos de Maior Duração:** Oferecer descontos e/ou benefícios progressivos ou exclusivos para aqueles que optarem por contratos de um ou dois anos.
*   **Problemas com Fibra Óptica:** Realizar uma análise sobre o serviço de fibra óptica tentando identificar e resolver problemas de qualidade, desempenho ou suporte que estejam contribuindo para a evasão.
*   **Método por Pagamento Cheque Eletrônico:** Verificar os problemas ou inconvenientes associados ao método de pagamento e considerar alternativas ou melhorias.
*   **Programa de Fidelidade para Clientes Iniciais:** Criar programas de fidelidade para os clientes nos primeiros meses de contrato aumentando a satisfação e reduzindo de evasão precoce.
*   **Clientes de Baixo Gasto:** Identificar os clientes com baixos gastos totais, com atenção especial para aqueles nos primeiros meses e entrar em contato para entender suas necessidades e oferecer soluções personalizadas.